<a href="https://colab.research.google.com/github/rho-selynn/592-HW3/blob/Roselynn/592_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# repo we're pushing/pulling to: https://github.com/rho-selynn/592-HW3
# repo mentioned in the homework: https://github.com/udacity/RoboND-Rover-Project
import os, sys
from google.colab import drive
# This mouunts your google drive to the current runtime
drive.mount('/content/mnt')
# We define a notebook path
nb_path = '/content/notebooks'
# We create a symbolic link from our drive's default "Colab Notebooks" folder to nb_path
os.symlink('/content/mnt/My Drive/Colab Notebooks/', nb_path)
# Insert nb path
sys.path.insert(0, nb_path)

Mounted at /content/mnt


In [5]:
# reference: https://github.com/ManajitPal/DeepLearningForSelfDrivingCars/blob/master/drive.py
# reference: https://github.com/hminle/car-behavioral-cloning-with-pytorch/
# reference: https://towardsdatascience.com/deep-learning-for-self-driving-cars-7f198ef4cfa2
# reference: https://arxiv.org/pdf/1604.07316v1.pdf 
# reference: https://github.com/abhinavsagar/Self-Driving-Car
# reference: https://towardsdatascience.com/how-to-train-your-self-driving-car-using-deep-learning-ce8ff76119cb

In [ ]:
!unzip '/content/mnt/My Drive/Colab Notebooks/RoverTrainingData_Grid.zip'

In [16]:
# importing dependencies
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

from torchvision import models

## Keras
import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam # had to add "tensorflow"
# reference: https://stackoverflow.com/questions/62707558/importerror-cannot-import-name-adam-from-keras-optimizers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense

## Sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import random
import ntpath

In [17]:
# Store data
datadir = 'RoverTrainingData_Grid'
columns = ['Path','SteerAngle', 'Throttle', 'Brake', 'Speed', 'X_Position', 'Y_Position', 'Pitch', 'Yaw', 'Roll']
data = pd.read_csv(os.path.join(datadir, 'robot_log.csv'), sep = ';', names = columns)
pd.set_option('display.max_colwidth', -1)
data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


,Path,SteerAngle,Throttle,Brake,Speed,X_Position,Y_Position,Pitch,Yaw,Roll
0,Path,SteerAngle,Throttle,Brake,Speed,X_Position,Y_Position,Pitch,Yaw,Roll
1,C:\Users\Roselynn Conrady\Documents\GitHub\592-HW3\RoverTrainingData_Grid\IMG\robocam_2022_03_23_15_22_10_540.jpg,0,0,1,0,99.66999,85.58897,0.0002357864,40.24681,-5.999252E-07
2,C:\Users\Roselynn Conrady\Documents\GitHub\592-HW3\RoverTrainingData_Grid\IMG\robocam_2022_03_23_15_22_10_597.jpg,0,0,0,0,99.66999,85.58897,0.0002357864,40.24681,-5.999252E-07
3,C:\Users\Roselynn Conrady\Documents\GitHub\592-HW3\RoverTrainingData_Grid\IMG\robocam_2022_03_23_15_22_10_671.jpg,0,0,0,0,99.66999,85.58897,0.0002357864,40.24681,-5.999252E-07
4,C:\Users\Roselynn Conrady\Documents\GitHub\592-HW3\RoverTrainingData_Grid\IMG\robocam_2022_03_23_15_22_10_732.jpg,0,0,0,0,99.66999,85.58897,0.0002357864,40.24681,-5.999252E-07
